<a href="https://colab.research.google.com/github/nolll77/LSTM_multiclass_text_classification/blob/master/LSTM_multiclass_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM in Pytorch

![Predicting item ratings based on customer reviews](https://drive.google.com/uc?id=1fi3BRAmlN7ljSnZLz2TuZmz9qZstZPxi)


Human language is filled with ambiguity, many-a-times the same phrase can have multiple interpretations based on the context and can even appear confusing to humans. Such challenges make natural language processing an interesting but hard problem to solve. However, we’ve seen a lot of advancement in NLP in the past couple of years and it’s quite fascinating to explore the various techniques being used. This article aims to cover one such technique in deep learning using Pytorch: Long Short Term Memory (LSTM) models.

If you’re new to NLP or need an in-depth read on preprocessing and word embeddings, you can check out the following [article](https://towardsdatascience.com/getting-started-with-natural-language-processing-nlp-2c482420cc05).

## Gentle Intro to RNNs and LSTMs :

What sets language models apart from conventional neural networks is their dependency on context. Conventional feed-forward networks assume inputs to be independent of one another. For NLP, we need a mechanism to be able to use sequential information from previous inputs to determine the current output. Recurrent Neural Networks (RNNs) tackle this problem by having loops, allowing information to persist through the network.

![An unrolled Recurrent Neural Network](https://drive.google.com/uc?id=1vFo5VFVHYGM4PTrKNVH3--iXJ-bNOD7u)

However, conventional RNNs have the issue of exploding and vanishing gradients and are not good at processing long sequences because they suffer from short term memory.

Long Short Term Memory networks (LSTM) are a special kind of RNN, which are capable of learning long-term dependencies. They do so by maintaining an internal memory state called the “cell state” and have regulators called “gates” to control the flow of information inside each LSTM unit. 

[Here](https://towardsdatascience.com/getting-started-with-natural-language-processing-nlp-2c482420cc05)’s an excellent source explaining the specifics of LSTMs.


![Structure of an LSTM cell. (source : Varsamopoulos, Savvas & Bertels, Koen & Almudever, Carmen. (2018). Designing neural network based decoders for surface codes.)
](https://drive.google.com/uc?id=1bXZ4F4OporY_NeanJ1m0sZMVMSBf8WJL)

In [ ]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

<IPython.core.display.Javascript object>

## Basic LSTM in Pytorch with random numbers

Before we jump into the main problem, let’s take a look at the basic structure of an LSTM in Pytorch, using a random input. This is a useful step to perform before getting into complex inputs because it helps us learn how to debug the model better, check if dimensions add up and ensure that our model is working as expected.

Even though we’re going to be dealing with text, since our model can only work with numbers, we convert the input into a sequence of numbers where each number represents a particular word (more on this in the next section).

In [ ]:
#input
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                 [12,45, 99,67, 6,23, 77,82],
                 [3,24, 6,99, 12,56, 21,22]])

#### using two different models

We first pass the input (3×8) through an [embedding](https://en.wikipedia.org/wiki/Word_embedding) layer, because word embeddings are better at capturing context and are spatially more efficient than one-hot vector representations.

In Pytorch, we can use the nn.Embedding module to create this layer, which takes the vocabulary size and desired word-vector length as input. You can optionally provide a padding index, to indicate the index of the padding element in the embedding matrix.

In the following example, our vocabulary consists of 100 words, so our input to the embedding layer can only be from 0–100, and it returns us a 100×7 embedding matrix, with the 0th index representing our padding element.

In [ ]:
model1 = nn.Embedding(100, 7, padding_idx=0)
model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

In [ ]:
out1 = model1(x)
out2 = model2(out1)

In [ ]:
print(out1.shape)
print(out1)

torch.Size([3, 8, 7])
tensor([[[-0.6816, -0.3904, -0.0229,  1.2287, -0.4489, -0.1448,  0.1876],
         [-0.1136, -0.2161, -0.6440,  1.7220, -1.0028, -0.4189,  1.4022],
         [ 0.1740, -0.2212,  1.5228, -1.1506, -1.1710,  0.0406, -0.3912],
         [ 1.9387,  0.3619, -0.4921, -0.6929, -0.6253,  1.1100,  0.8697],
         [-1.1030,  0.5688, -0.2015, -1.0526,  2.9643,  1.2638,  1.9368],
         [-0.3143, -0.8116, -0.1972,  0.9615,  0.8048, -0.2469,  1.0350],
         [ 0.2626, -0.4890, -1.0185,  0.4583,  0.6501, -0.1358,  0.1586],
         [-0.4704,  1.3602,  0.6796,  0.4018, -0.2171,  2.0806, -1.0199]],

        [[ 0.1740, -0.2212,  1.5228, -1.1506, -1.1710,  0.0406, -0.3912],
         [-0.9500,  0.7904,  0.0888, -1.0316, -0.7365, -0.8333,  0.6342],
         [-1.0811,  0.2237, -0.4557,  0.4708,  0.8445, -1.0519,  0.0446],
         [ 1.3037, -1.0439, -0.8036,  0.5445,  1.7022,  0.7845,  0.0318],
         [ 0.8269, -0.6542, -0.3596,  1.8055, -0.8318,  0.6261,  0.2298],
         [-0.2

##### We pass the embedding layer’s output into an LSTM layer (created using nn.LSTM), which takes as input the word-vector length, length of the hidden state vector and number of layers. Additionally, if the first element in our input’s shape has the batch size, we can specify batch_first = True

The LSTM layer outputs three things:

The consolidated output — of all hidden states in the sequence

*   The consolidated output — of all hidden states in the sequence
*   Hidden state of the last LSTM unit — the final output
*   Cell state

Hidden state of the last LSTM unit — the final output
Cell state


We can verify that after passing through all layers, our output has the expected dimensions:

3×8 -> embedding -> 3x8x7 -> LSTM (with hidden size=3)-> 3×3

In [ ]:
out, (ht, ct) = model2(out1)
print(ht)

tensor([[[ 0.2788,  0.1724,  0.0143],
         [-0.2855,  0.1841,  0.2382],
         [-0.5367,  0.0766,  0.0969]]], grad_fn=<StackBackward>)


#### using nn.sequential

In [ ]:
model3 = nn.Sequential(nn.Embedding(100, 7, padding_idx=0),
                        nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True))

In [ ]:
out, (ht, ct) = model3(x)
print(out)

tensor([[[-0.2137,  0.0261, -0.1449],
         [-0.3795, -0.0441, -0.0928],
         [-0.3562, -0.0813, -0.0430],
         [-0.3664, -0.0105, -0.2085],
         [-0.5796, -0.0787, -0.3419],
         [-0.4169, -0.1716, -0.3466],
         [-0.4096, -0.0531, -0.5017],
         [-0.6825, -0.0244, -0.3172]],

        [[-0.1501, -0.0516, -0.0249],
         [ 0.0922, -0.0493, -0.0280],
         [ 0.1433, -0.1797, -0.0193],
         [ 0.3654, -0.1216, -0.0491],
         [ 0.4690, -0.1234, -0.0121],
         [ 0.3195, -0.0079, -0.1690],
         [ 0.2842, -0.0074, -0.2765],
         [ 0.0661, -0.0064, -0.2403]],

        [[-0.2451, -0.1169, -0.2873],
         [-0.3352, -0.0299, -0.0743],
         [-0.0965, -0.0810, -0.1175],
         [ 0.0841, -0.2123, -0.0219],
         [-0.0312, -0.2138, -0.0325],
         [-0.2179, -0.1398, -0.2310],
         [-0.1087, -0.0455, -0.0574],
         [-0.1863, -0.1855, -0.1135]]], grad_fn=<TransposeBackward0>)


## Multiclass Text Classification - Predicting ratings from review comments

Let’s now look at an application of LSTMs.

Problem Statement: Given an item’s review comment, predict the rating ( takes integer values from 1 to 5, 1 being worst and 5 being best)

We are going to predict item ratings based on customer reviews bsed on this dataset from Kaggle :

[Here](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews) or [here](https://towardsdatascience.com/getting-started-with-natural-language-processing-nlp-2c482420cc05)

In [ ]:
#loading the data
reviews = pd.read_csv("reviews.csv")
print(reviews.shape)
reviews.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [ ]:
#keeping only relevant columns and calculating sentence lengths
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews['review_length'] = reviews['review'].apply(lambda x: len(x.split()))
reviews.head()

,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,4,8
1,Love this dress! it's sooo pretty. i happen...,5,62
2,Some major design flaws I had such high hopes ...,3,102
3,"My favorite buy! I love, love, love this jumps...",5,25
4,Flattering shirt This shirt is very flattering...,5,38


In [ ]:
#changing ratings to 0-numbering
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
reviews['rating'] = reviews['rating'].apply(lambda x: zero_numbering[x])

In [ ]:
#mean sentence length
np.mean(reviews['review_length'])

60.832921740611425

#### Metric
We usually take accuracy as our metric for most classification problems, however, ratings are ordered. If the actual value is 5 but the model predicts a 4, it is not considered as bad as predicting a 1. Hence, instead of going with accuracy, we choose RMSE — root mean squared error as our North Star metric. Also, rating prediction is a pretty hard problem, even for humans, so a prediction of being off by just 1 point or lesser is considered pretty good.

Preprocessing
As mentioned earlier, we need to convert our text into a numerical form that can be fed to our model as input. I’ve used spacy for tokenization after removing punctuation, special characters, and lower casing the text:

In [ ]:
#tokenization
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

#### We count the number of occurrences of each token in our corpus and get rid of the ones that don’t occur too frequently :

In [ ]:
#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['review']))

#### We lost about 6000 words! This is expected because our corpus is quite small, less than 25k reviews, the chance of having repeated words is quite small.

We then create a vocabulary to index mapping and encode our review text using this mapping. I’ve chosen the maximum length of any review to be 70 words because the average length of reviews was around 60.

In [ ]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 14138
num_words after: 8263


In [ ]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
reviews.head()

,review,rating,review_length,encoded
0,Absolutely wonderful - silky and sexy and com...,3,8,"[[2, 3, 4, 5, 6, 7, 8, 7, 9, 0, 0, 0, 0, 0, 0,..."
1,Love this dress! it's sooo pretty. i happen...,4,62,"[[2, 10, 11, 12, 5, 13, 14, 15, 16, 5, 17, 18,..."
2,Some major design flaws I had such high hopes ...,2,102,"[[54, 55, 56, 57, 17, 58, 59, 60, 61, 62, 11, ..."
3,"My favorite buy! I love, love, love this jumps...",4,25,"[[68, 109, 110, 2, 17, 10, 2, 10, 2, 10, 11, 1..."
4,Flattering shirt This shirt is very flattering...,4,38,"[[122, 123, 11, 123, 52, 92, 122, 19, 124, 125..."


In [ ]:
#check how balanced the dataset is
Counter(reviews['rating'])

Counter({3: 5077, 4: 13131, 2: 2871, 1: 1565, 0: 842})

In [ ]:
X = list(reviews['encoded'])
y = list(reviews['rating'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

#### Pytorch Dataset

The dataset is quite straightforward because we’ve already stored our encodings in the input dataframe. We also output the length of the input sequence in each case, because we can have LSTMs that take variable-length sequences.

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

#### Pytorch training loop
The training loop is pretty standard. I’ve used Adam optimizer and cross-entropy loss.



In [ ]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

## LSTM Model
I’ve used 3 variations for the model:

### 1- LSTM with fixed length input

This pretty much has the same structure as the basic LSTM we saw earlier, with the addition of a dropout layer to prevent overfitting. Since we have a classification problem, we have a final linear layer with 5 outputs. This implementation actually works the best among the classification LSTMs, with an accuracy of about 64% and a root-mean-squared-error of only 0.817

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 1.239, val loss 1.218, val accuracy 0.556, and val rmse 1.355
train loss 1.188, val loss 1.203, val accuracy 0.554, and val rmse 1.348
train loss 1.112, val loss 1.165, val accuracy 0.574, and val rmse 1.189
train loss 1.072, val loss 1.161, val accuracy 0.517, and val rmse 1.145
train loss 1.042, val loss 1.117, val accuracy 0.563, and val rmse 1.279
train loss 0.981, val loss 1.113, val accuracy 0.572, and val rmse 1.216


In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 0.936, val loss 1.065, val accuracy 0.576, and val rmse 1.177
train loss 0.846, val loss 1.009, val accuracy 0.603, and val rmse 0.937
train loss 0.848, val loss 1.009, val accuracy 0.606, and val rmse 0.912
train loss 0.784, val loss 0.994, val accuracy 0.604, and val rmse 0.894
train loss 0.741, val loss 0.984, val accuracy 0.617, and val rmse 0.870
train loss 0.702, val loss 0.999, val accuracy 0.623, and val rmse 0.863


In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 0.695, val loss 1.010, val accuracy 0.619, and val rmse 0.869
train loss 0.630, val loss 0.992, val accuracy 0.625, and val rmse 0.836
train loss 0.583, val loss 1.020, val accuracy 0.632, and val rmse 0.823
train loss 0.545, val loss 1.052, val accuracy 0.635, and val rmse 0.823
train loss 0.502, val loss 1.088, val accuracy 0.634, and val rmse 0.827
train loss 0.469, val loss 1.145, val accuracy 0.639, and val rmse 0.817


### 2- LSTM with variable length input

We can modify our model a bit to make it accept variable-length inputs. This ends up increasing the training time though, because of the pack_padded_sequence function call which returns a padded batch of variable-length sequences.

In [ ]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [ ]:
model = LSTM_variable_input(vocab_size, 50, 50)

In [ ]:
train_model(model, epochs=30, lr=0.1)

train loss 1.328, val loss 1.250, val accuracy 0.515, and val rmse 1.312
train loss 1.031, val loss 1.063, val accuracy 0.577, and val rmse 1.017
train loss 0.904, val loss 0.995, val accuracy 0.603, and val rmse 0.941
train loss 0.849, val loss 1.000, val accuracy 0.599, and val rmse 0.940
train loss 0.845, val loss 1.009, val accuracy 0.598, and val rmse 0.921
train loss 0.834, val loss 1.005, val accuracy 0.593, and val rmse 0.902


In [ ]:
train_model(model, epochs=30, lr=0.05)

train loss 0.828, val loss 1.000, val accuracy 0.599, and val rmse 0.920
train loss 0.790, val loss 0.989, val accuracy 0.605, and val rmse 0.894
train loss 0.775, val loss 0.992, val accuracy 0.614, and val rmse 0.884
train loss 0.755, val loss 0.994, val accuracy 0.597, and val rmse 0.883
train loss 0.738, val loss 0.987, val accuracy 0.608, and val rmse 0.872
train loss 0.741, val loss 1.005, val accuracy 0.611, and val rmse 0.888


In [ ]:
train_model(model, epochs=30, lr=0.05)

train loss 0.758, val loss 1.028, val accuracy 0.616, and val rmse 0.884
train loss 0.725, val loss 0.994, val accuracy 0.621, and val rmse 0.877
train loss 0.715, val loss 0.999, val accuracy 0.607, and val rmse 0.881
train loss 0.707, val loss 1.008, val accuracy 0.608, and val rmse 0.879
train loss 0.698, val loss 1.018, val accuracy 0.615, and val rmse 0.890
train loss 0.686, val loss 1.017, val accuracy 0.603, and val rmse 0.893


### 3- LSTM with pretrained Glove word embeddings

Instead of training our own word embeddings, we can use pre-trained Glove word vectors that have been trained on a massive corpus and probably have better context captured. For our problem, however, this doesn’t seem to help much.

Download weights [here](https://nlp.stanford.edu/projects/glove/)

In [ ]:
def load_glove_vectors(glove_file="./data/glove.6B/glove.6B.50d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [ ]:
def get_emb_matrix(pretrained, word_counts, emb_size = 50):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [ ]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [ ]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model = LSTM_glove_vecs(vocab_size, 50, 50, pretrained_weights)

In [ ]:
train_model(model, epochs=30, lr=0.1)

train loss 1.281, val loss 1.255, val accuracy 0.556, and val rmse 1.355
train loss 1.210, val loss 1.207, val accuracy 0.556, and val rmse 1.354
train loss 1.206, val loss 1.204, val accuracy 0.556, and val rmse 1.354
train loss 1.201, val loss 1.202, val accuracy 0.556, and val rmse 1.354
train loss 1.173, val loss 1.168, val accuracy 0.557, and val rmse 1.352
train loss 1.131, val loss 1.122, val accuracy 0.562, and val rmse 1.249


In [ ]:
train_model(model, epochs=30, lr=0.05)

train loss 1.112, val loss 1.113, val accuracy 0.556, and val rmse 1.349
train loss 1.061, val loss 1.051, val accuracy 0.570, and val rmse 1.109
train loss 1.014, val loss 1.014, val accuracy 0.582, and val rmse 1.058
train loss 0.979, val loss 0.990, val accuracy 0.599, and val rmse 0.995
train loss 0.948, val loss 0.961, val accuracy 0.610, and val rmse 0.950
train loss 0.923, val loss 0.952, val accuracy 0.612, and val rmse 0.935


In [ ]:
train_model(model, epochs=30, lr=0.05)

train loss 1.189, val loss 1.014, val accuracy 0.586, and val rmse 1.033
train loss 0.946, val loss 0.964, val accuracy 0.606, and val rmse 0.950
train loss 0.912, val loss 0.951, val accuracy 0.612, and val rmse 0.941
train loss 0.895, val loss 0.949, val accuracy 0.615, and val rmse 0.913
train loss 0.886, val loss 0.947, val accuracy 0.617, and val rmse 0.901
train loss 0.872, val loss 0.938, val accuracy 0.621, and val rmse 0.890


## Predicting ratings using regression instead of classification

Since ratings have an order, and a prediction of 3.6 might be better than rounding off to 4 in many cases, it is helpful to explore this as a regression problem. Not surprisingly, this approach gives us the lowest error of just 0.799 because we don’t have just integer predictions anymore.

The only change to our model is that instead of the final layer having 5 outputs, we have just one. The training loop changes a bit too, we use MSE loss and we don’t need to take the argmax anymore to get the final prediction.

In [ ]:
def train_model_regr(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.float()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.mse_loss(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss = validation_metrics_regr(model, val_dl)
        if i % 5 == 1:
            print("train mse %.3f val rmse %.3f" % (sum_loss/total, val_loss))

def validation_metrics_regr (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.float()
        y_hat = model(x, l)
        loss = np.sqrt(F.mse_loss(y_hat, y.unsqueeze(-1)).item())
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total

In [ ]:
class LSTM_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model =  LSTM_regr(vocab_size, 50, 50)

In [ ]:
train_model_regr(model, epochs=30, lr=0.05)

train mse 1.663 val rmse 1.313
train mse 1.215 val rmse 1.125
train mse 1.151 val rmse 1.109
train mse 1.114 val rmse 1.115
train mse 1.082 val rmse 1.121
train mse 1.043 val rmse 1.116


In [ ]:
train_model_regr(model, epochs=30, lr=0.05)

train mse 1.214 val rmse 1.193
train mse 0.884 val rmse 1.032
train mse 0.631 val rmse 0.903
train mse 0.483 val rmse 0.837
train mse 0.416 val rmse 0.806
train mse 0.363 val rmse 0.799


## Conclusion:
LSTM appears to be theoretically involved, but its Pytorch implementation is pretty straightforward. Also, while looking at any problem, it is very important to choose the right metric, in our case if we’d gone for accuracy, the model seems to be doing a very bad job, but the RMSE shows that it is off by less than 1 rating point, which is comparable to human performance!

References:

https://www.usfca.edu/data-institute/certificates/deep-learning-part-one
https://colah.github.io/posts/2015-08-Understanding-LSTMs/
http://web.stanford.edu/class/cs224n/